In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [ ]:
PATH = "data/mckinsey/"

In [ ]:
!ls {PATH}

In [ ]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False, 
                     parse_dates=["DOB","Lead_Creation_Date"])


In [1]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [ ]:
display_all(df_raw.tail().transpose())

In [ ]:
display_all(df_raw.describe(include='all').transpose())

In [ ]:
len(df_raw[df_raw.Approved==1])

In [ ]:
train_cats(df_raw)

In [2]:
def basic_date_part(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year','Month'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)


Convert cats to codes

In [ ]:
basic_date_part(df_raw,'DOB')
basic_date_part(df_raw,'Lead_Creation_Date')
df_raw = df_raw.drop('ID',axis=1)
df_raw = df_raw.drop('DOBMonth',axis=1)
df_raw = df_raw.drop('DOBYear',axis=1)

In [ ]:
df, y, nas = proc_df(df_raw, 'Approved')

In [ ]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/mckinsey-raw')

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
PATH = "data/mckinsey/"

In [4]:

df_raw = pd.read_feather('tmp/mckinsey-raw')
len(df_raw)

69713

In [5]:
n_valid = 10  # same as Kaggle's test set size
n_trn = len(df_raw)-n_valid

In [6]:
train_cats(df_raw)

In [7]:
# df_raw['avg']= df_raw.apply(lambda row: (row.Monthly_Income - row.Existing_EMI) / row.EMI, axis=1)

In [8]:
df, y, nas = proc_df(df_raw, 'Approved')

In [9]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)


In [10]:

display_all(df.tail().transpose())

# df = df.drop('Existing_EMI_na',axis=1)

,69708,69709,69710,69711,69712
Gender,1,1,1,2,2
City_Code,6,116,22,2,3
City_Category,1,3,2,1,1
Employer_Code,9,30250,11756,97,55
Employer_Category1,1,1,3,3,1
Employer_Category2,1,4,4,3,1
Monthly_Income,4900,7190.1,1600,9893,4230
Customer_Existing_Primary_Bank_Code,2,2,30,2,0
Primary_Bank_Type,2,2,2,2,0
Contacted,1,1,2,2,2


In [11]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()


raw_train, raw_valid = split_vals(df_raw, n_valid)
X_valid, X_train = split_vals(df, n_valid)
y_valid, y_train = split_vals(y, n_valid)

X_train.shape, y_train.shape, X_valid.shape

((69703, 28), (69703,), (10, 28))

In [12]:
len(y_train[y_train==1])

1020

Things to try
1) Check how it improves wrt to number of trees in forest
2) set samples n check
3) Check min leaves
4) Check max features
5) Do feature importance and remove unwanted rows

In [13]:
# preds = np.stack([t.predict(X_valid) for t in m.estimators_])
# preds[:,0], np.mean(preds[:,0]), y_valid[0]

In [14]:
# preds.shape

In [15]:
# plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(40)]);

1) 40 Trees is as good as we get

In [462]:
set_rf_samples(6000)
m = RandomForestRegressor(n_estimators=7000,max_features=0.15,n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.15, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=7000, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [463]:
# fi = rf_feat_importance(m, X_train); fi[:30]

In [443]:
fi = rf_feat_importance(m, X_train); fi[:30]

,cols,imp
18,DOBElapsed,0.103045
3,Employer_Code,0.098998
6,Monthly_Income,0.098646
21,Lead_Creation_Elapsed,0.085055
1,City_Code,0.069353
12,Existing_EMI,0.069208
13,Loan_Amount,0.061306
7,Customer_Existing_Primary_Bank_Code,0.059295
20,Lead_Creation_Month,0.049267
16,EMI,0.045528


In [289]:
y_pred = m.predict(X_valid)



In [290]:
# for i in range(len(y_pred)):
#     if(X_valid.iloc[i]['Contacted']==1 and X_valid.iloc[i]['Gender']==2):
#         y_pred[i]=0
    

In [377]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_valid,y_pred)

In [ ]:
plt.plot(y_pred)

In [ ]:
y_pred[y_pred>=0.5]=1
y_pred[y_pred<0.5]=0

Check Confusion matrix

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid,y_pred)

In [ ]:
len(y_pred[y_pred==1])

In [ ]:
y_pred_int_type = y_pred.astype(int)

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_pred_int_type,y_valid,average='micro')

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred_int_type,y_valid)



In [ ]:
df_to_print = pd.DataFrame()
for i in range(len(X_valid)):
    if(y_pred_int_type[i]==0 and y_valid[i]==1):
        df_to_print = df_to_print.append(X_valid.iloc[i])
len(df_to_print)

In [ ]:
# fpr, tpr, thresholds = metrics.roc_curve(y_valid, y_pred)

# plt.plot(fpr, tpr)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.rcParams['font.size'] = 12
# plt.title('ROC curve')
# plt.xlabel('False Positive Rate (1 - Specificity)')
# plt.ylabel('True Positive Rate (Sensitivity)')
# plt.grid(True)

In [ ]:
fi = rf_feat_importance(m, X_train); fi[:30]

In [ ]:
def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [ ]:
plot_fi(fi[:30]);

In [ ]:
to_keep = fi[fi.imp>0.005].cols; len(to_keep)

In [ ]:
df_keep = df[to_keep].copy()
X_train, X_valid = split_vals(df_keep, n_trn)

In [ ]:
reset_rf_samples()
m = RandomForestRegressor(n_estimators=350,max_features=0.5,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)

In [ ]:
y_pred = m.predict(X_valid)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid,y_pred)

In [464]:
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False, 
                     parse_dates=["DOB","Lead_Creation_Date"])

In [465]:
basic_date_part(df_test,'Lead_Creation_Date')
basic_date_part(df_test,'DOB')


In [466]:
df_test = df_test.drop('ID',axis=1)
df_test = df_test.drop('DOBMonth',axis=1)
df_test = df_test.drop('DOBYear',axis=1)

In [467]:
train_cats(df_test)

In [468]:
y_temp = np.zeros(len(df_test))
y_temp.shape


(30037,)

In [469]:
df_test['Approved']=y_temp

In [470]:
# display_all(df_test.head())

In [471]:
df_test_for_model,ytemp,nas = proc_df(df_test,'Approved')

In [472]:
# df_test_to_keep = df_test_for_model[to_keep].copy()

In [473]:
# display_all(df_test_to_keep.head())

In [474]:
# count=0
# for i in range(len(df_test_for_model)):
#     if(df_test_for_model.iloc[i]['Gender']==2 and df_test_for_model.iloc[i]['Contacted']==2):
#         count+=1
# count

In [475]:
y_final_pred = m.predict(df_test_for_model)

In [476]:
y_final_pred[:10]

array([0.03029, 0.08971, 0.01543, 0.01886, 0.02414, 0.01457, 0.05514, 0.02143, 0.06471, 0.10043])

In [477]:
ids = pd.read_csv(f'{PATH}test.csv', low_memory=False).ID

In [478]:
subm = np.stack([ids,y_final_pred], axis=1)


In [479]:
subm[:5]

array([['APPA70109647212', 0.030285714285714287],
       ['APPB10687939341', 0.08971428571428572],
       ['APPC80449411414', 0.015428571428571429],
       ['APPD30665094501', 0.018857142857142857],
       ['APPE80379821637', 0.024142857142857143]], dtype=object)

In [480]:
submission_file_name = 'mckinsey_6.csv'
np.savetxt(submission_file_name, subm, fmt='%s,%.5f', header='ID,Approved', comments='')

In [481]:
from IPython.display import FileLink
FileLink(submission_file_name)

/Users/gokulakrishnanm/Desktop/Deeplearning/contests/analytics-vidhya/coursetemp/fastai/courses/ml1/mckinsey_6.csv